In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv("201606-citibike-tripdata.csv")
df.head()

In [ ]:
#df.iloc[0]
#for i in df.iloc[0]:
    #print(type(i))
    
df['starttime'] = pd.to_datetime(df.starttime)
df['stoptime'] = pd.to_datetime(df.stoptime)

df.head()

meantime = df.groupby(['start station id', 'end station id'])['tripduration'].mean().reset_index()
stdtime = df.groupby(['start station id', 'end station id'])['tripduration'].std().reset_index()

combinetime = pd.merge(meantime, stdtime, how='inner', on=['start station id', 'end station id'])
combinetime

tripcount = df.groupby(['start station id', 'end station id'])['tripduration'].count().reset_index()

tripcount

In [ ]:
ended_station=df[['end station id', 'bikeid', 'stoptime']]
cols1 = ['station id', 'bikeid', 'stoptime']
ended_station.columns = cols1
ended_station.head()

In [ ]:
start_station =df[['start station id', 'bikeid', 'starttime']]
cols2 = ['station id', 'bikeid', 'starttime']
start_station.columns = cols2
start_station.head()

In [ ]:
# Create a dataframe that contains where the bike parks, when tt got parked  and when it got riden away
mergedata = pd.merge(ended_station, start_station, how='left', on = ['station id','bikeid'])
mergedata.head()

In [ ]:
mergedata['TimeDiff'] = mergedata['starttime'] - mergedata['stoptime']
mergedata.head()

In [ ]:
list(mergedata)
#mergedata.head()

In [ ]:
mergedata.dtypes

In [ ]:
mergedata = mergedata.drop(mergedata[mergedata['TimeDiff'] < pd.Timedelta(0)].index)
mergedata = mergedata[mergedata.TimeDiff.notnull()]


In [ ]:
mergedata_clean = mergedata.groupby(['station id', 'bikeid', 'stoptime'])['TimeDiff'].min().reset_index()

mergedata_clean

In [ ]:
mergedata_clean['TimeDiff_convert'] = mergedata_clean['TimeDiff'].apply(lambda row: row.total_seconds())

mergedata_clean.head()

In [ ]:
mergedata_clean.dtypes

In [ ]:
mean_station_time = mergedata_clean.groupby(['station id'])['TimeDiff_convert'].median().reset_index()
mean_station_time

In [ ]:
mean_station_time.to_csv('mean_station_time.csv', index=False)